In [1]:
import tensorflow as tf
import os
import numpy as np
import scipy as sp
from tqdm import tqdm_notebook
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from inspect import signature
from PIL import Image
from scipy.cluster.vq import kmeans
import matplotlib.pyplot as plt
from multiprocessing import Pool
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ReduceLROnPlateau
from decimal import *
from numpy import dot
from numpy.linalg import norm



# folder where all *.npy files are located
path_to_data = r'C:\Users\Lenovo\Downloads\Sem_3\RP\Labelling_softmax\files'

# create log files in `current dir/`
logs = 'logs/' 

reference_features = np.load(os.path.join(path_to_data, 'training_features_62_classes.npy'))
reference_features = reference_features / np.linalg.norm(reference_features, 2, axis=1, keepdims=True)
reference_classes = np.load(os.path.join(path_to_data, 'training_classids_62_classes.npy'))
reference_probs = np.load(os.path.join(path_to_data, 'training_probabilities_62_classes.npy'))

incremental_features = np.load(os.path.join(path_to_data, 'validation_features_62_classes.npy'))

incremental_features = incremental_features / np.linalg.norm(incremental_features, 2 , axis=1, keepdims=True)
incremental_classes = np.load(os.path.join(path_to_data, 'validation_classids_62_classes.npy'))
incremental_probs = np.load(os.path.join(path_to_data, 'validation_probabilities_62_classes.npy'))

test_features = np.load(os.path.join(path_to_data, 'test_features_62_classes.npy'))
test_features = test_features / np.linalg.norm(test_features, 2 , axis=1, keepdims=True)
test_classes = np.load(os.path.join(path_to_data, 'test_classids_62_classes.npy'))
test_probs = np.load(os.path.join(path_to_data, 'test_probabilities_62_classes.npy'))

class_mapping = np.load(os.path.join(path_to_data, 'class_mapping_62_classes.npy'), allow_pickle=True).item()
unique_classes = np.unique(reference_classes)

test_dataset = tf.data.Dataset.from_tensor_slices((test_features, tf.keras.utils.to_categorical(test_classes)))
test_dataset = test_dataset.batch(32)


In [14]:
print(reference_features.shape)
print(incremental_features.shape)
print(test_features.shape)
print(reference_features.max())

(40603, 2048)
(17401, 2048)
(31000, 2048)
0.88654864


In [80]:

def eucl_distance(x,features):
    diff = x - features          
    dist = np.sqrt(np.square(diff).sum(axis=-1))
    return dist

def soft_distance(x, features, gamma):
    diff = x - features          
    dist = np.sqrt(np.square(diff).sum(axis=-1))
    return np.exp(-gamma * dist).sum()

def calculate_distances(features, anchor, gamma):
    labels = np.zeros((features.shape[0],))
    probs = np.zeros_like(labels)
    for i, feature in enumerate(features): 
        distances = np.zeros((len(unique_classes),))
        for cid in unique_classes:            
            n = soft_distance(feature, anchor[cid], gamma)            
            ds = np.finfo(float).eps
            for ccid in unique_classes:                
                d = soft_distance(feature, anchor[ccid], gamma)
                ds += d
            
            distances[cid] = n/ds
            
        labels[i] = distances.argmax()
        probs[i]  = distances.max()
    return labels, probs

def get_soft_voting(features, anchor, gamma):       
    labels = []
    probs = []
    cpus = os.cpu_count() - 2 # keep two cpus please
    num_chunks = features.shape[0] // (cpus - 1)
    
    with Pool(processes=cpus) as pool:
        results = []

        for chunk in range(cpus):
            start = chunk * num_chunks
            
            if chunk == cpus - 1:
                sub_features = features[start:]
            else:
                sub_features = features[start:start + num_chunks]
            
            results.append(pool.apply_async(calculate_distances, args=(sub_features, anchor, gamma)))
        
        with tqdm_notebook(total=features.shape[0]) as pbar:        
            for result in results:
                label, prob = result.get()
                labels.extend(label)
                probs.extend(prob)
                pbar.update(len(label))
        
    return labels, probs

def get_anchor_points(num_points):
    anchor_points = []
    for cid in tqdm_notebook(unique_classes, desc='calculate anchor points'):
        indices = np.where(reference_classes == cid)[0]
        class_features = reference_features[indices]
        codebook, _ = kmeans(class_features, num_points, iter=200)
        #codebook /= np.linalg.norm(codebook, 2, axis=1, keepdims=True)
        
        anchor_points += [codebook]
    return np.array(anchor_points)

def calculate_weights(features, name):
    if name == 'baseline':
        return np.ones(features.shape[0])
    if name == 'weighted_prob':
        weight_prob = np.zeros_like(probs)
        weight_prob = np.float(1.0) - (probs.max(axis=1))
        return np.array(weight_prob)
    if name == 'active_0.1':
        prob_test=(1.0-np.array(probs.max(axis=1)))
        prob_test_new = []
        index = []
        for i in range(len(prob_test)):
            if prob_test[i] >= 0.1:
                prob_test_new.append((prob_test[i]))
                index.append(i)
        return np.array(prob_test_new), np.array(index)
    if name == 'active_0.3':
        prob_test=(1.0-np.array(probs.max(axis=1)))
        prob_test_new = []
        index = []
        for i in range(len(prob_test)):
            if prob_test[i] >= 0.3:
                prob_test_new.append((prob_test[i]))
                index.append(i)
        return np.array(prob_test_new), np.array(index)
    if name == 'active_0.5':
        prob_test=(1.0-np.array(probs.max(axis=1)))
        prob_test_new = []
        index = []
        for i in range(len(prob_test)):
            if prob_test[i] >= 0.5:
                prob_test_new.append((prob_test[i]))
                index.append(i)
        return np.array(prob_test_new), np.array(index)
    if name == 'active_0.7':
        prob_test=(1.0-np.array(probs.max(axis=1)))
        prob_test_new = []
        index = []
        for i in range(len(prob_test)):
            if prob_test[i] >= 0.7:
                prob_test_new.append((prob_test[i]))
                index.append(i)
        return np.array(prob_test_new), np.array(index)
    if name == 'active_0.9':
        prob_test=(1.0-np.array(probs.max(axis=1)))
        prob_test_new = []
        index = []
        for i in range(len(prob_test)):
            if prob_test[i] >= 0.9:
                prob_test_new.append((prob_test[i]))
                index.append(i)
        return np.array(prob_test_new), np.array(index)

In [3]:
anchors = np.load('anchor_points.npy')

In [4]:
labels_soft = np.load('labels_l2.npy')
probs = np.load('prob_l2.npy')

In [41]:
np.save('labels_gamma10',labels_l2)
np.save('probs_gamma10',prob_l2)

In [111]:
labels_soft=incremental_probs.argmax(-1)
probs=incremental_probs
print(np.array(probs).max())

1.0


In [75]:
t = []
for i in range(len(incremental_features)):
    t_1 = []
    for j in range(62):
        t_2 = []
        for k in range(10):
            y= cosine_dist(incremental_features[i],anchors[j,k,:],1.5)
            t_2+=[y]
        t_1+=[t_2]
    t+=[t_1]
print(np.array(t).shape)

(17401, 62, 10)


In [76]:
print(np.array(t))

[[[0.40700662 0.60008472 0.55102077 ... 0.4565239  0.59312296 0.69113448]
  [0.44033653 0.42703801 0.38895518 ... 0.7360574  0.78532252 0.46443748]
  [0.81436101 0.83488175 0.86050227 ... 0.81945911 0.83160858 0.82052197]
  ...
  [0.70854023 0.68446732 0.71513122 ... 0.73259747 0.66332841 0.68682861]
  [0.89529008 0.7082231  0.83129407 ... 0.82205689 0.85139193 0.80672376]
  [0.77823991 0.83042416 0.78349556 ... 0.82337011 0.78567642 0.79287495]]

 [[0.63051149 0.61036608 0.64928779 ... 0.57506666 0.61129844 0.71632928]
  [0.61151841 0.62852392 0.61809009 ... 0.80901884 0.86785939 0.6829344 ]
  [0.87880749 0.89076098 0.91807322 ... 0.82928783 0.88934181 0.87794321]
  ...
  [0.78305027 0.74316707 0.76482731 ... 0.76303206 0.75656243 0.71086645]
  [0.83221117 0.64716047 0.76614054 ... 0.79186626 0.73225778 0.7607199 ]
  [0.80873032 0.86091274 0.82931185 ... 0.84894027 0.79351149 0.81687872]]

 [[0.71531892 0.74250728 0.73242742 ... 0.60236919 0.75421934 0.04336387]
  [0.68513268 0.732793

In [37]:
dist_prob =[]
for i in range(len(incremental_features)):
    for i_anch in range(62):
        dist_class = anchors[int(i_anch),:,:]
        anch = []
        for anch_point in dist_class:
            n_d = sp.spatial.distance.cdist(np.expand_dims(incremental_features[i],0), np.expand_dims(anch_point,0), 'cosine')
            anch += [n_d]
    dist_prob += [anch]

In [38]:
print(dist_prob,np.array(dist_prob).shape)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [29]:
res_new=[]
for feat in incremental_features:
    for anch in range(62):
        x = sp.spatial.distance.cdist(incremental_features[i], anch_point, 'cosine')
        res_new += [x]

print(np.array(res_new).shape)

(1078862, 1, 10)


In [110]:
print(labels_soft = incremental_classes)

In [23]:
x_1,y_1=calculate_weights(incremental_features,'weighted_prob')
print(x_1.shape)

NameError: name 'probs' is not defined

In [7]:
print(incremental_features[0].shape)
print(anchors[0][0].shape)

(2048,)
(2048,)


In [117]:
np.array(prob_l2).max()

0.14274030570384727

In [112]:
accu = accuracy_score(incremental_classes, labels_soft )
print(accu)

0.6671455663467617


In [113]:
accu_softmax = accuracy_score(incremental_classes, incremental_probs.argmax(-1) )
print(accu_softmax)

0.6671455663467617


In [36]:
np.save('labels_l2',labels_l2)

In [17]:
anchors.shape


(62, 10, 2048)

In [18]:
labels_soft

array([ 0,  0,  0, ..., 61, 61, 61], dtype=int32)

In [89]:
np.linalg.norm(reference_features[2])

0.9999999

In [84]:
for class_anchor in anchors:
    print(np.linalg.norm(class_anchor, 2, axis = 0).)

[0.01451669 0.0122645  0.01339182 ... 0.14653212 0.01101872 0.00648832]
[0.01320241 0.02109883 0.01426366 ... 0.10881281 0.00408078 0.01051635]
[0.10007218 0.01597041 0.00659267 ... 0.01942493 0.03644577 0.01108169]
[0.03927026 0.01624916 0.11495394 ... 0.00742108 0.04862261 0.03737556]
[0.00926008 0.01142468 0.06746539 ... 0.00596215 0.02744677 0.0235248 ]
[0.02082544 0.01939648 0.02740207 ... 0.01579915 0.02335838 0.01513518]
[0.0338123  0.01029657 0.00697266 ... 0.04726903 0.00979435 0.01131043]
[0.18779463 0.0452951  0.01080569 ... 0.03966454 0.01137339 0.00897109]
[0.03809591 0.06821112 0.01449743 ... 0.02022204 0.01284392 0.02335776]
[0.0128266  0.0075739  0.02177125 ... 0.0096603  0.06007257 0.01894998]
[0.01152607 0.0266415  0.05527271 ... 0.11630904 0.01775121 0.01554709]
[0.04419288 0.0161972  0.00943661 ... 0.02089878 0.06922931 0.04984759]
[0.00848678 0.01385924 0.01591597 ... 0.00713631 0.0196746  0.01517598]
[0.01773687 0.03229105 0.07495635 ... 0.0065653  0.05520876 0.01

In [82]:

weig= calculate_weights(incremental_features,'weighted_prob')
print(weig)

[0.2870627  0.6543143  0.00142133 ... 0.07339907 0.00238127 0.00073856]


In [107]:
1-(probs[3].max())

0.2014973759651184

In [83]:

prob_test_new,ind = calculate_weights(incremental_features,'active_0.1')

In [86]:

print(prob_test_new)

[0.2870627  0.6543143  0.20149738 ... 0.492535   0.19299942 0.54772735]
[    0     1     3 ... 17386 17389 17397]


In [57]:
prob_test=(1.0-np.array(probs))
prob_test_new = []
index = []
for i in range(len(prob_test)):
    if prob_test[i] >= 0.1:
        prob_test_new.append((prob_test[i]))
        index.append(i)
        

In [114]:
labesl_soft

(17401,)

In [8]:
len(incremental_classes[index])

12

In [ ]:
experiment_weights_types = ['baseline']
for name in experiment_weights_types:
    for attempt in range(3):
        # sample weights 
        weights = calculate_weights(incremental_features, name)

        incremental_dataset = tf.data.Dataset.from_tensor_slices((incremental_features, tf.keras.utils.to_categorical(labels_soft),weights))
        incremental_dataset = incremental_dataset.shuffle(2048).batch(32)
        # load pre-trained keras model
        model = tf.keras.models.load_model(os.path.join(path_to_data, 'submodel_62_classes.hdf5'), compile=False)

        # use SGD optimizer
        optimizer = optimizers.Adam(lr=.0001)
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # define callbacks 
        callbacks = [
            ModelCheckpoint(
                os.path.join(logs, '{}_model_62_classes.hdf5'.format(name)), monitor='val_accuracy', 
                verbose=0, save_best_only=True, save_weights_only=False, mode='auto'),
            # you can use tensorboard --logdir path/to/log/dir to visualize the training parameters like loss and accuracy
            TensorBoard(os.path.join(logs, 'sm_'+name, str(attempt))),
            #ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,patience=10, min_lr=0.000001, verbose=1)

        ] 

        # start training for 250 epochs
        model.fit_generator(
            incremental_dataset,
            epochs=500,
            callbacks=callbacks,
            validation_data=test_dataset)

        model.load_weights(os.path.join(logs, '{}_model_62_classes.hdf5'.format(name)))
        
        y_pred = model.predict(test_dataset)
        cm_b = confusion_matrix(test_classes, np.array(y_pred).argmax(axis=1))
        
        np.save(os.path.join('cm_sm_'+name+'_'+str(attempt)),cm_b)
        pred_accu = accuracy_score(test_classes, np.array(y_pred).argmax(axis=1) )
        print(os.path.join('cm_sm_'+name+'_'+str(attempt)),pred_accu)


Epoch 1/500
544/544 [==============================] - 5s 10ms/step - loss: 3.7941 - accuracy: 0.5188 - val_loss: 3.7441 - val_accuracy: 0.3468
Epoch 2/500
544/544 [==============================] - 5s 10ms/step - loss: 3.5826 - accuracy: 0.4196 - val_loss: 3.5797 - val_accuracy: 0.3306
Epoch 3/500
544/544 [==============================] - 5s 10ms/step - loss: 3.3947 - accuracy: 0.4279 - val_loss: 3.4234 - val_accuracy: 0.3577
Epoch 4/500
544/544 [==============================] - 5s 10ms/step - loss: 3.2201 - accuracy: 0.4712 - val_loss: 3.2737 - val_accuracy: 0.4004
Epoch 5/500
544/544 [==============================] - 5s 10ms/step - loss: 3.0562 - accuracy: 0.5220 - val_loss: 3.1304 - val_accuracy: 0.4398
Epoch 6/500
544/544 [==============================] - 5s 10ms/step - loss: 2.9014 - accuracy: 0.5713 - val_loss: 2.9933 - val_accuracy: 0.4723
Epoch 7/500
544/544 [==============================] - 5s 10ms/step - loss: 2.7555 - accuracy: 0.6096 - val_loss: 2.8624 - val_accuracy:

544/544 [==============================] - 5s 10ms/step - loss: 0.3811 - accuracy: 0.9192 - val_loss: 0.9796 - val_accuracy: 0.7078
Epoch 114/500
544/544 [==============================] - 5s 10ms/step - loss: 0.3786 - accuracy: 0.9197 - val_loss: 0.9800 - val_accuracy: 0.7077
Epoch 115/500
544/544 [==============================] - 5s 9ms/step - loss: 0.3763 - accuracy: 0.9199 - val_loss: 0.9806 - val_accuracy: 0.7077
Epoch 116/500
544/544 [==============================] - 5s 10ms/step - loss: 0.3740 - accuracy: 0.9206 - val_loss: 0.9809 - val_accuracy: 0.7077
Epoch 117/500
544/544 [==============================] - 5s 10ms/step - loss: 0.3717 - accuracy: 0.9209 - val_loss: 0.9812 - val_accuracy: 0.7075
Epoch 118/500
544/544 [==============================] - 5s 10ms/step - loss: 0.3695 - accuracy: 0.9213 - val_loss: 0.9817 - val_accuracy: 0.7074
Epoch 119/500
544/544 [==============================] - 5s 10ms/step - loss: 0.3673 - accuracy: 0.9225 - val_loss: 0.9822 - val_accuracy: 

544/544 [==============================] - 5s 10ms/step - loss: 0.2228 - accuracy: 0.9654 - val_loss: 1.0655 - val_accuracy: 0.7020
Epoch 226/500
544/544 [==============================] - 5s 10ms/step - loss: 0.2220 - accuracy: 0.9652 - val_loss: 1.0663 - val_accuracy: 0.7018
Epoch 227/500
544/544 [==============================] - 5s 10ms/step - loss: 0.2211 - accuracy: 0.9659 - val_loss: 1.0672 - val_accuracy: 0.7018
Epoch 228/500
544/544 [==============================] - 5s 10ms/step - loss: 0.2202 - accuracy: 0.9660 - val_loss: 1.0680 - val_accuracy: 0.7019
Epoch 229/500
544/544 [==============================] - 5s 10ms/step - loss: 0.2194 - accuracy: 0.9663 - val_loss: 1.0689 - val_accuracy: 0.7019
Epoch 230/500
544/544 [==============================] - 5s 10ms/step - loss: 0.2184 - accuracy: 0.9665 - val_loss: 1.0699 - val_accuracy: 0.7018
Epoch 231/500
544/544 [==============================] - 5s 10ms/step - loss: 0.2176 - accuracy: 0.9669 - val_loss: 1.0704 - val_accuracy:

544/544 [==============================] - 5s 9ms/step - loss: 0.1490 - accuracy: 0.9856 - val_loss: 1.1634 - val_accuracy: 0.6992
Epoch 338/500
544/544 [==============================] - 5s 10ms/step - loss: 0.1485 - accuracy: 0.9857 - val_loss: 1.1642 - val_accuracy: 0.6993
Epoch 339/500
544/544 [==============================] - 5s 9ms/step - loss: 0.1480 - accuracy: 0.9858 - val_loss: 1.1649 - val_accuracy: 0.6992
Epoch 340/500
544/544 [==============================] - 5s 10ms/step - loss: 0.1476 - accuracy: 0.9859 - val_loss: 1.1658 - val_accuracy: 0.6992
Epoch 341/500
544/544 [==============================] - 5s 9ms/step - loss: 0.1471 - accuracy: 0.9862 - val_loss: 1.1667 - val_accuracy: 0.6991
Epoch 342/500
544/544 [==============================] - 5s 9ms/step - loss: 0.1466 - accuracy: 0.9863 - val_loss: 1.1675 - val_accuracy: 0.6992
Epoch 343/500
544/544 [==============================] - 5s 10ms/step - loss: 0.1461 - accuracy: 0.9864 - val_loss: 1.1685 - val_accuracy: 0.6

544/544 [==============================] - 5s 9ms/step - loss: 0.1048 - accuracy: 0.9943 - val_loss: 1.2573 - val_accuracy: 0.6965
Epoch 450/500
544/544 [==============================] - 5s 10ms/step - loss: 0.1045 - accuracy: 0.9943 - val_loss: 1.2581 - val_accuracy: 0.6965
Epoch 451/500
544/544 [==============================] - 5s 10ms/step - loss: 0.1042 - accuracy: 0.9944 - val_loss: 1.2588 - val_accuracy: 0.6965
Epoch 452/500
544/544 [==============================] - 5s 10ms/step - loss: 0.1039 - accuracy: 0.9943 - val_loss: 1.2596 - val_accuracy: 0.6966
Epoch 453/500
544/544 [==============================] - 5s 10ms/step - loss: 0.1035 - accuracy: 0.9944 - val_loss: 1.2604 - val_accuracy: 0.6966
Epoch 454/500
544/544 [==============================] - 5s 10ms/step - loss: 0.1032 - accuracy: 0.9944 - val_loss: 1.2616 - val_accuracy: 0.6965
Epoch 455/500
544/544 [==============================] - 5s 10ms/step - loss: 0.1029 - accuracy: 0.9945 - val_loss: 1.2624 - val_accuracy: 

In [ ]:
np.ones(incremental_features.shape[0]).shape

In [20]:
y_pred = model.predict(test_dataset)
    
cm_b = confusion_matrix(test_classes, np.array(y_pred).argmax(axis=1))

In [21]:
print(np.array(y_pred).argmax(axis=1))

[ 0  0 26 ... 61 61  7]


In [ ]:
print(test_classes)

In [ ]:
print(cm)

In [22]:
pred_accu = accuracy_score(test_classes, np.array(y_pred).argmax(axis=1) )
print(pred_accu)

0.7978387096774193


In [23]:
np.save(os.path.join('cm_probablity_baseline_'+str(attempt)),cm_b)

In [68]:
np.save('cm_probablity_baseline_1',cm_b)

In [83]:
experiment_weights_types = ['active_0.1','active_0.3','active_0.5','active_0.7','active_0.9']
for name in experiment_weights_types:
    for attempt in range(3):
        # sample weights 
        weights,ind = calculate_weights(incremental_features, name)

        incremental_dataset = tf.data.Dataset.from_tensor_slices((incremental_features[ind], tf.keras.utils.to_categorical(incremental_classes[ind]),weights))
        incremental_dataset = incremental_dataset.shuffle(2048).batch(32)
        # load pre-trained keras model
        model = tf.keras.models.load_model(os.path.join(path_to_data, 'submodel_62_classes.hdf5'), compile=False)

        # use SGD optimizer
        optimizer = optimizers.Adam(lr=.0001)
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # define callbacks 
        callbacks = [
            ModelCheckpoint(
                os.path.join(logs, '{}_model_62_classes.hdf5'.format(name)), monitor='val_accuracy', 
                verbose=0, save_best_only=True, save_weights_only=False, mode='auto'),
            # you can use tensorboard --logdir path/to/log/dir to visualize the training parameters like loss and accuracy
            TensorBoard(os.path.join(logs,'l2_'+name,str(attempt))),
            #ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,patience=10, min_lr=0.000001, verbose=1)

        ] 

        # start training for 250 epochs
        model.fit_generator(
            incremental_dataset,
            epochs=500,
            callbacks=callbacks,
            validation_data=test_dataset)

        model.load_weights(os.path.join(logs, '{}_model_62_classes.hdf5'.format(name)))
        
        y_pred = model.predict(test_dataset)
        cm_b = confusion_matrix(test_classes, np.array(y_pred).argmax(axis=1))
        
        np.save(os.path.join('cm_l2_'+name+'_'+str(attempt)),cm_b)
        pred_accu = accuracy_score(test_classes, np.array(y_pred).argmax(axis=1) )
        print(os.path.join('cm_l2_'+name+'_'+str(attempt)),pred_accu)
        
        

Epoch 1/500
538/538 [==============================] - 5s 10ms/step - loss: 2.9298 - accuracy: 0.3063 - val_loss: 3.8008 - val_accuracy: 0.5241
Epoch 2/500
538/538 [==============================] - 5s 10ms/step - loss: 2.7813 - accuracy: 0.4224 - val_loss: 3.6439 - val_accuracy: 0.6289
Epoch 3/500
538/538 [==============================] - 5s 10ms/step - loss: 2.6663 - accuracy: 0.5259 - val_loss: 3.4961 - val_accuracy: 0.6814
Epoch 4/500
538/538 [==============================] - 5s 10ms/step - loss: 2.5586 - accuracy: 0.6059 - val_loss: 3.3551 - val_accuracy: 0.7045
Epoch 5/500
538/538 [==============================] - 6s 10ms/step - loss: 2.4567 - accuracy: 0.6608 - val_loss: 3.2204 - val_accuracy: 0.7181
Epoch 6/500
538/538 [==============================] - 5s 10ms/step - loss: 2.3599 - accuracy: 0.6960 - val_loss: 3.0914 - val_accuracy: 0.7257
Epoch 7/500
538/538 [==============================] - 5s 10ms/step - loss: 2.2677 - accuracy: 0.7181 - val_loss: 2.9680 - val_accuracy:

538/538 [==============================] - 5s 9ms/step - loss: 0.4653 - accuracy: 0.8606 - val_loss: 0.6608 - val_accuracy: 0.8193
Epoch 114/500
538/538 [==============================] - 5s 10ms/step - loss: 0.4630 - accuracy: 0.8634 - val_loss: 0.6591 - val_accuracy: 0.8195
Epoch 115/500
538/538 [==============================] - 5s 10ms/step - loss: 0.4610 - accuracy: 0.8626 - val_loss: 0.6574 - val_accuracy: 0.8197
Epoch 116/500
538/538 [==============================] - 5s 10ms/step - loss: 0.4589 - accuracy: 0.8633 - val_loss: 0.6558 - val_accuracy: 0.8197
Epoch 117/500
538/538 [==============================] - 6s 10ms/step - loss: 0.4569 - accuracy: 0.8627 - val_loss: 0.6542 - val_accuracy: 0.8198
Epoch 118/500
538/538 [==============================] - 5s 10ms/step - loss: 0.4549 - accuracy: 0.8638 - val_loss: 0.6526 - val_accuracy: 0.8197
Epoch 119/500
538/538 [==============================] - 5s 10ms/step - loss: 0.4529 - accuracy: 0.8648 - val_loss: 0.6510 - val_accuracy: 

538/538 [==============================] - 5s 10ms/step - loss: 0.3198 - accuracy: 0.8994 - val_loss: 0.5722 - val_accuracy: 0.8281
Epoch 226/500
538/538 [==============================] - 5s 10ms/step - loss: 0.3189 - accuracy: 0.9008 - val_loss: 0.5719 - val_accuracy: 0.8282
Epoch 227/500
538/538 [==============================] - 5s 9ms/step - loss: 0.3181 - accuracy: 0.9000 - val_loss: 0.5715 - val_accuracy: 0.8282
Epoch 228/500
538/538 [==============================] - 5s 10ms/step - loss: 0.3173 - accuracy: 0.9007 - val_loss: 0.5712 - val_accuracy: 0.8284
Epoch 229/500
538/538 [==============================] - 5s 10ms/step - loss: 0.3164 - accuracy: 0.9002 - val_loss: 0.5708 - val_accuracy: 0.8282
Epoch 230/500
538/538 [==============================] - 5s 10ms/step - loss: 0.3155 - accuracy: 0.9008 - val_loss: 0.5705 - val_accuracy: 0.8285
Epoch 231/500
538/538 [==============================] - 5s 10ms/step - loss: 0.3147 - accuracy: 0.9020 - val_loss: 0.5701 - val_accuracy: 

538/538 [==============================] - 5s 10ms/step - loss: 0.2432 - accuracy: 0.9269 - val_loss: 0.5495 - val_accuracy: 0.8301
Epoch 338/500
538/538 [==============================] - 5s 10ms/step - loss: 0.2426 - accuracy: 0.9270 - val_loss: 0.5494 - val_accuracy: 0.8301
Epoch 339/500
538/538 [==============================] - 5s 10ms/step - loss: 0.2421 - accuracy: 0.9280 - val_loss: 0.5493 - val_accuracy: 0.8301
Epoch 340/500
538/538 [==============================] - 6s 11ms/step - loss: 0.2415 - accuracy: 0.9274 - val_loss: 0.5492 - val_accuracy: 0.8301
Epoch 341/500
538/538 [==============================] - 5s 10ms/step - loss: 0.2410 - accuracy: 0.9284 - val_loss: 0.5491 - val_accuracy: 0.8300
Epoch 342/500
538/538 [==============================] - 5s 10ms/step - loss: 0.2405 - accuracy: 0.9277 - val_loss: 0.5490 - val_accuracy: 0.8301
Epoch 343/500
538/538 [==============================] - 5s 10ms/step - loss: 0.2400 - accuracy: 0.9281 - val_loss: 0.5489 - val_accuracy:

538/538 [==============================] - 5s 10ms/step - loss: 0.1911 - accuracy: 0.9470 - val_loss: 0.5431 - val_accuracy: 0.8318
Epoch 450/500
538/538 [==============================] - 5s 10ms/step - loss: 0.1906 - accuracy: 0.9474 - val_loss: 0.5431 - val_accuracy: 0.8319
Epoch 451/500
538/538 [==============================] - 5s 10ms/step - loss: 0.1903 - accuracy: 0.9474 - val_loss: 0.5431 - val_accuracy: 0.8321
Epoch 452/500
538/538 [==============================] - 5s 10ms/step - loss: 0.1899 - accuracy: 0.9474 - val_loss: 0.5431 - val_accuracy: 0.8321
Epoch 453/500
538/538 [==============================] - 5s 10ms/step - loss: 0.1895 - accuracy: 0.9475 - val_loss: 0.5430 - val_accuracy: 0.8321
Epoch 454/500
538/538 [==============================] - 5s 10ms/step - loss: 0.1891 - accuracy: 0.9476 - val_loss: 0.5430 - val_accuracy: 0.8321
Epoch 455/500
538/538 [==============================] - 5s 10ms/step - loss: 0.1887 - accuracy: 0.9484 - val_loss: 0.5430 - val_accuracy:

...

129/129 [==============================] - 3s 24ms/step - loss: 3.6974 - accuracy: 0.3125 - val_loss: 3.8784 - val_accuracy: 0.6060
Epoch 2/500
129/129 [==============================] - 3s 23ms/step - loss: 3.6440 - accuracy: 0.3147 - val_loss: 3.8466 - val_accuracy: 0.4909
Epoch 3/500
129/129 [==============================] - 3s 23ms/step - loss: 3.5982 - accuracy: 0.2875 - val_loss: 3.8170 - val_accuracy: 0.3945
Epoch 4/500
129/129 [==============================] - 3s 24ms/step - loss: 3.5563 - accuracy: 0.2641 - val_loss: 3.7891 - val_accuracy: 0.3273
Epoch 5/500
129/129 [==============================] - 3s 24ms/step - loss: 3.5178 - accuracy: 0.2493 - val_loss: 3.7623 - val_accuracy: 0.2869
Epoch 6/500
129/129 [==============================] - 3s 23ms/step - loss: 3.4820 - accuracy: 0.2384 - val_loss: 3.7363 - val_accuracy: 0.2594
Epoch 7/500
129/129 [==============================] - 3s 24ms/step - loss: 3.4484 - accuracy: 0.2379 - val_loss: 3.7110 - val_accuracy: 0.2445
Epoc

129/129 [==============================] - 3s 23ms/step - loss: 1.8018 - accuracy: 0.6267 - val_loss: 1.8890 - val_accuracy: 0.6125
Epoch 114/500
129/129 [==============================] - 3s 24ms/step - loss: 1.7932 - accuracy: 0.6303 - val_loss: 1.8793 - val_accuracy: 0.6138
Epoch 115/500
129/129 [==============================] - 3s 23ms/step - loss: 1.7848 - accuracy: 0.6298 - val_loss: 1.8696 - val_accuracy: 0.6158
Epoch 116/500
129/129 [==============================] - 3s 24ms/step - loss: 1.7764 - accuracy: 0.6337 - val_loss: 1.8600 - val_accuracy: 0.6175
Epoch 117/500
129/129 [==============================] - 3s 24ms/step - loss: 1.7681 - accuracy: 0.6361 - val_loss: 1.8507 - val_accuracy: 0.6190
Epoch 118/500
129/129 [==============================] - 3s 24ms/step - loss: 1.7598 - accuracy: 0.6378 - val_loss: 1.8411 - val_accuracy: 0.6207
Epoch 119/500
129/129 [==============================] - 3s 23ms/step - loss: 1.7517 - accuracy: 0.6383 - val_loss: 1.8316 - val_accuracy:

129/129 [==============================] - 3s 23ms/step - loss: 1.1660 - accuracy: 0.7682 - val_loss: 1.2138 - val_accuracy: 0.7214
Epoch 226/500
129/129 [==============================] - 3s 23ms/step - loss: 1.1623 - accuracy: 0.7670 - val_loss: 1.2106 - val_accuracy: 0.7219
Epoch 227/500
129/129 [==============================] - 3s 23ms/step - loss: 1.1585 - accuracy: 0.7679 - val_loss: 1.2069 - val_accuracy: 0.7225
Epoch 228/500
129/129 [==============================] - 3s 23ms/step - loss: 1.1549 - accuracy: 0.7682 - val_loss: 1.2036 - val_accuracy: 0.7228
Epoch 229/500
129/129 [==============================] - 3s 23ms/step - loss: 1.1512 - accuracy: 0.7689 - val_loss: 1.2002 - val_accuracy: 0.7233
Epoch 230/500
129/129 [==============================] - 3s 24ms/step - loss: 1.1476 - accuracy: 0.7701 - val_loss: 1.1969 - val_accuracy: 0.7235
Epoch 231/500
129/129 [==============================] - 3s 24ms/step - loss: 1.1439 - accuracy: 0.7718 - val_loss: 1.1937 - val_accuracy:

129/129 [==============================] - 3s 24ms/step - loss: 0.8487 - accuracy: 0.8377 - val_loss: 0.9636 - val_accuracy: 0.7511
Epoch 338/500
129/129 [==============================] - 3s 23ms/step - loss: 0.8465 - accuracy: 0.8387 - val_loss: 0.9623 - val_accuracy: 0.7510
Epoch 339/500
129/129 [==============================] - 3s 23ms/step - loss: 0.8443 - accuracy: 0.8396 - val_loss: 0.9609 - val_accuracy: 0.7513
Epoch 340/500
129/129 [==============================] - 3s 24ms/step - loss: 0.8422 - accuracy: 0.8399 - val_loss: 0.9596 - val_accuracy: 0.7513
Epoch 341/500
129/129 [==============================] - 3s 23ms/step - loss: 0.8401 - accuracy: 0.8406 - val_loss: 0.9581 - val_accuracy: 0.7517
Epoch 342/500
129/129 [==============================] - 3s 23ms/step - loss: 0.8379 - accuracy: 0.8404 - val_loss: 0.9569 - val_accuracy: 0.7518
Epoch 343/500
129/129 [==============================] - 3s 24ms/step - loss: 0.8358 - accuracy: 0.8418 - val_loss: 0.9554 - val_accuracy:

129/129 [==============================] - 3s 24ms/step - loss: 0.6500 - accuracy: 0.8847 - val_loss: 0.8543 - val_accuracy: 0.7629
Epoch 450/500
129/129 [==============================] - 3s 23ms/step - loss: 0.6485 - accuracy: 0.8842 - val_loss: 0.8534 - val_accuracy: 0.7630
Epoch 451/500
129/129 [==============================] - 3s 23ms/step - loss: 0.6471 - accuracy: 0.8852 - val_loss: 0.8528 - val_accuracy: 0.7631
Epoch 452/500
129/129 [==============================] - 3s 23ms/step - loss: 0.6456 - accuracy: 0.8857 - val_loss: 0.8521 - val_accuracy: 0.7632
Epoch 453/500
129/129 [==============================] - 3s 23ms/step - loss: 0.6442 - accuracy: 0.8861 - val_loss: 0.8515 - val_accuracy: 0.7632
Epoch 454/500
129/129 [==============================] - 3s 23ms/step - loss: 0.6428 - accuracy: 0.8866 - val_loss: 0.8510 - val_accuracy: 0.7631
Epoch 455/500
129/129 [==============================] - 3s 23ms/step - loss: 0.6413 - accuracy: 0.8871 - val_loss: 0.8501 - val_accuracy:

In [14]:
y_pred_x = model.predict(test_dataset)
    
cm_x = confusion_matrix(test_classes, np.array(y_pred_x).argmax(axis=1))

In [15]:
pred_accu = accuracy_score(test_classes, np.array(y_pred_x).argmax(axis=1) )
print(pred_accu)

0.8170967741935484


In [16]:
np.save('cm_probablity_ac01_1',cm_x)

In [5]:
import numpy as np
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

import matplotlib as mpl
import matplotlib.pyplot as plt

def plot_tensorflow_log(path):

    # Loading too much data is slow...
    tf_size_guidance = {
        'compressedHistograms': 10,
        'images': 0,
        'scalars': 100,
        'histograms': 1
    }

    event_acc = EventAccumulator(path, tf_size_guidance)
    event_acc.Reload()

    # Show all tags in the log file
    #print(event_acc.Tags())

    #training_accuracies =   event_acc.Scalars('training-accuracy')
    validation_accuracies = event_acc.Scalars('validation_accuracy')

    steps = 500
    x = np.arange(steps)
    y = np.zeros([steps, 2])

    for i in xrange(steps):
        #y[i, 0] = training_accuracies[i][2] # value
        y[i, 1] = validation_accuracies[i][2]

    plt.plot(x, y[:,0], label='training accuracy')
    plt.plot(x, y[:,1], label='validation accuracy')

    plt.xlabel("Steps")
    plt.ylabel("Accuracy")
    plt.title("Training Progress")
    plt.legend(loc='upper right', frameon=True)
    plt.show()


if __name__ == '__main__':
    log_file = r"C:\Users\Lenovo\Downloads\Sem_3\RP\logs\sm_baseline\0\validation\events.out.tfevents.1579560452.dst-aux-ws-11.14279.209418674.v2"
    plot_tensorflow_log(log_file)

KeyError: 'Key validation_accuracy was not found in Reservoir'

In [7]:
for event in tf.backend.summary_iterator(r"C:\Users\Lenovo\Downloads\Sem_3\RP\logs\sm_baseline\0\validation\events.out.tfevents.1579560452.dst-aux-ws-11.14279.209418674.v2"):
    for value in event.summary.value:
        print(value.tag)
        if value.HasField('simple_value'):
            print(value.simple_value)

AttributeError: module 'tensorflow' has no attribute 'backend'